In [ ]:
%pip install google-cloud-storage
%pip install pandas
%pip install numpy
%pip install fsspec
%pip install gcsfs

In [1]:
from google.cloud import storage
import pandas as pd
import numpy as np

In [2]:
bucket_name = 'ppp-loan-analysis'
gs_path  = f'gs://{bucket_name}/'
raw_folder = 'raw-data/'
clean_folder = 'clean-data/'
final_folder = 'final-data/'
dropped_folder = 'dropped-data/'
gdp_folder = 'gdp-data/'

In [3]:
storage_client = storage.Client() 
bucket = storage_client.get_bucket(bucket_name)

# Raw GDP Data

In [4]:
raw_gdp_path = gs_path + raw_folder + gdp_folder + 'CAGDP1__ALL_AREAS_2001_2023.csv'
raw_gdp_df = pd.read_csv(raw_gdp_path, header=0)

In [ ]:
raw_gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9538 entries, 0 to 9537
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   GeoFIPS                 9538 non-null   object 
 1   GeoName                 9534 non-null   object 
 2   Region                  9534 non-null   object 
 3   TableName               9534 non-null   object 
 4   LineCode                9534 non-null   float64
 5   IndustryClassification  9534 non-null   object 
 6   Description             9534 non-null   object 
 7   Unit                    9534 non-null   object 
 8   2001                    9534 non-null   object 
 9   2002                    9534 non-null   object 
 10  2003                    9534 non-null   object 
 11  2004                    9534 non-null   object 
 12  2005                    9534 non-null   object 
 13  2006                    9534 non-null   object 
 14  2007                    9534 non-null   

In [7]:
raw_gdp_df.describe(include='all')

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
count,9538,9534,9534,9534,9534.000000,9534,9534,9534,9534,9534,...,9534,9534,9534,9534,9534,9534,9534,9534,9534,9534
unique,3182,3178,9,1,NaN,1,3,3,9419,9438,...,9378,9352,9270,3172,9190,9291,9349,9379,9398,9397
top,"""00000""",United States,5,CAGDP1,NaN,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,(NA),(NA),...,(NA),(NA),(NA),100.000,(NA),(NA),(NA),(NA),(NA),(NA)
freq,3,3,3144,9534,NaN,9534,3178,3178,24,21,...,15,15,15,3173,15,15,12,12,12,12
mean,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,0.816539,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Year cols: 2001 - 2023
year_cols = [str(year) for year in range(2001, 2024)]
# Convert year columns to float
raw_gdp_df[year_cols] = raw_gdp_df[year_cols].astype(float)
# Summary statistics for year columns
raw_gdp_df[year_cols].describe()


,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
count,9.507000e+03,9.510000e+03,9.510000e+03,9.510000e+03,9.510000e+03,9.510000e+03,9.510000e+03,9.514000e+03,9.519000e+03,9.519000e+03,...,9.519000e+03,9.519000e+03,9.519000e+03,9.519000e+03,9.519000e+03,9.519000e+03,9.520000e+03,9.520000e+03,9.520000e+03,9.520000e+03
mean,1.041074e+07,1.065576e+07,1.104484e+07,1.160475e+07,1.217556e+07,1.268653e+07,1.309927e+07,1.322186e+07,1.290458e+07,1.332944e+07,...,1.500913e+07,1.552504e+07,1.587931e+07,1.641666e+07,1.709074e+07,1.767845e+07,1.740587e+07,1.889997e+07,2.009416e+07,2.107986e+07
std,1.998688e+08,2.043519e+08,2.115793e+08,2.219146e+08,2.324856e+08,2.419497e+08,2.495173e+08,2.517328e+08,2.458547e+08,2.538221e+08,...,2.856718e+08,2.955442e+08,3.024268e+08,3.127436e+08,3.257669e+08,3.371309e+08,3.321316e+08,3.609052e+08,3.845099e+08,4.039535e+08
min,3.051000e+00,3.234000e+00,3.279000e+00,3.159000e+00,3.042000e+00,2.898000e+00,3.084000e+00,3.197000e+00,3.503000e+00,4.977000e+00,...,2.035800e+01,3.370900e+01,4.740500e+01,1.000000e+02,5.760300e+01,4.853800e+01,4.797100e+01,4.447300e+01,4.299000e+01,3.405100e+01
25%,9.275900e+01,9.483025e+01,9.644750e+01,9.919650e+01,1.009808e+02,1.019355e+02,1.016037e+02,1.022638e+02,1.010630e+02,1.036260e+02,...,1.042285e+02,1.041425e+02,1.026070e+02,1.000000e+02,1.043120e+02,1.068385e+02,1.061560e+02,1.118063e+02,1.140888e+02,1.186995e+02
50%,2.557040e+05,2.663520e+05,2.774040e+05,2.934165e+05,3.041690e+05,3.158380e+05,3.243300e+05,3.332165e+05,3.311150e+05,3.459480e+05,...,3.829890e+05,3.829070e+05,3.802580e+05,3.789210e+05,3.903530e+05,3.998120e+05,3.945080e+05,4.335660e+05,4.586295e+05,4.745420e+05
75%,1.124880e+06,1.171031e+06,1.205425e+06,1.272415e+06,1.319486e+06,1.373379e+06,1.407182e+06,1.447202e+06,1.405144e+06,1.457109e+06,...,1.621423e+06,1.637601e+06,1.640226e+06,1.678499e+06,1.726940e+06,1.762001e+06,1.708948e+06,1.852178e+06,1.987203e+06,2.059488e+06
max,1.423073e+10,1.447271e+10,1.487731e+10,1.544976e+10,1.598796e+10,1.643315e+10,1.676244e+10,1.678148e+10,1.634911e+10,1.678975e+10,...,1.826171e+10,1.879962e+10,1.914167e+10,1.961210e+10,2.065652e+10,2.153998e+10,2.135410e+10,2.368117e+10,2.600689e+10,2.772071e+10


In [11]:
# Summary statistics for the entire year columns (i.e., treat all year values as one column)
raw_gdp_df[year_cols].stack().describe()

count    2.188700e+05
mean     1.471837e+07
std      2.862585e+08
min      2.898000e+00
25%      1.048200e+02
50%      3.513480e+05
75%      1.525846e+06
max      2.772071e+10
dtype: float64

In [19]:
# Print the total number of rows in the dataframe
total_rows = len(raw_gdp_df)
print(f"Total number of rows: {total_rows}")

Total number of rows: 9538


In [20]:
# Print the number of unique GeoFIPS in the dataframe
unique_geofips = raw_gdp_df['GeoFIPS'].nunique()
print(f"Number of unique GeoFIPS: {unique_geofips}")

Number of unique GeoFIPS: 3182


In [ ]:
# Show the first 10 GEOFIPS values
print("First 10 GeoFIPS values:")
print(raw_gdp_df['GeoFIPS'][:10])

First 10 GeoFIPS values:
0     "00000"
1     "00000"
2     "00000"
3     "01000"
4     "01000"
5     "01000"
6     "01001"
7     "01001"
8     "01001"
9     "01003"
Name: GeoFIPS, dtype: object


In [6]:
# Create a new dataframe with the unique GeoFIPS values and how many times they appear in the original dataframe
geofips_counts = raw_gdp_df['GeoFIPS'].value_counts().reset_index()
geofips_counts.columns = ['GeoFIPS', 'count']
# Order GeoFIPS by count descending
geofips_counts = geofips_counts.sort_values(by='count', ascending=True)
geofips_counts.head(10)

,GeoFIPS,count
3181,U.S. Bureau of Economic Analysis,1
3180,"Last updated: December 4, 2024-- new statistic...",1
3179,CAGDP1: County and MSA gross domestic product ...,1
3178,Note: See the included footnote file.,1
3171,"""92000""",3
3170,"""91000""",3
3169,"""56045""",3
3168,"""56043""",3
3167,"""56041""",3
3166,"""56039""",3


In [ ]:
# Unique values in the Region column
print("Unique values in the Region column:")
print(raw_gdp_df['Region'].unique())

Unique values in the Region column:
[' ' '5' '8' '6' '7' '1' '2' '3' '4' nan]


In [ ]:
# Number of columns that have ' ' as a value in the Region column
num_blank_region = (raw_gdp_df['Region'] == ' ').sum()
print(f"Number of rows with blank Region: {num_blank_region}")
# Rows with ' ' as a value in the Region column
raw_gdp_df[raw_gdp_df['Region'] == ' ']

Number of rows with blank Region: 3


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"""00000""",United States,,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,1.423073e+10,1.447271e+10,...,1.826171e+10,1.879962e+10,1.914167e+10,1.961210e+10,2.019390e+10,2.071567e+10,2.026758e+10,2.149480e+10,2.203483e+10,2.267110e+10
1,"""00000""",United States,,CAGDP1,2.0,...,Chain-type quantity indexes for real GDP,Quantity index,7.256100e+01,7.379500e+01,...,9.311500e+01,9.585700e+01,9.760100e+01,1.000000e+02,1.029670e+02,1.056270e+02,1.033420e+02,1.096000e+02,1.123530e+02,1.155970e+02
2,"""00000""",United States,,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,1.058193e+10,1.092911e+10,...,1.760814e+10,1.829502e+10,1.880491e+10,1.961210e+10,2.065652e+10,2.153998e+10,2.135410e+10,2.368117e+10,2.600689e+10,2.772071e+10


# Clean GDP Data

In [4]:
clean_gdp_path = gs_path + clean_folder + gdp_folder + 'cleaned_gdp_data.csv'
clean_gdp_df = pd.read_csv(clean_gdp_path, header=0)

In [ ]:
# Schema
clean_gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19032 entries, 0 to 19031
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   facts_gdp_id          19032 non-null  int64  
 1   geofips               19032 non-null  int64  
 2   geo_name              19032 non-null  object 
 3   region                19032 non-null  object 
 4   year_id               19032 non-null  int64  
 5   chain_type_index_gdp  19032 non-null  float64
 6   current_dollar_gdp    19032 non-null  float64
 7   real_gdp              19032 non-null  float64
dtypes: float64(3), int64(3), object(2)
memory usage: 1.2+ MB


In [ ]:
# Nmber of rows in the cleaned dataframe
clean_total_rows = len(clean_gdp_df)
print(f"Total number of rows in cleaned dataframe: {clean_total_rows}")

Total number of rows in cleaned dataframe: 19032


In [ ]:
# Number of unique GeoFIPS in the dataframe
clean_unique_geo_fips = clean_gdp_df['geofips'].nunique()
print(f"Number of unique GeoFIPS in cleaned dataframe: {clean_unique_geo_fips}")

Number of unique GeoFIPS in cleaned dataframe: 3172


In [ ]:
# Number of unique values each column
clean_gdp_df.nunique()

facts_gdp_id            19032
geofips                  3172
geo_name                 3172
region                      9
year_id                     6
chain_type_index_gdp    12394
current_dollar_gdp      18971
real_gdp                18965
dtype: int64

In [20]:
# Summary statistics for: chain_type_index_gdp, current_dollar_gdp, real_gdp
clean_gdp_df[['chain_type_index_gdp', 'current_dollar_gdp', 'real_gdp']].describe()

,chain_type_index_gdp,current_dollar_gdp,real_gdp
count,19032.000000,1.903200e+04,1.903200e+04
mean,103.504111,2.780089e+07,2.601069e+07
std,12.277503,4.343273e+08,4.049614e+08
min,42.990000,9.855000e+03,8.792000e+03
25%,99.421750,4.253210e+05,3.930378e+05
50%,101.207000,1.127444e+06,1.040029e+06
75%,106.943250,3.609289e+06,3.331000e+06
max,382.776000,2.600689e+10,2.203483e+10


In [10]:
# Check if there is a record with geofips = 23003.0
clean_gdp_df[clean_gdp_df['geofips'] == 23003.0].count()

facts_gdp_id            6
geofips                 6
geo_name                6
region                  6
year_id                 6
chain_type_index_gdp    6
current_dollar_gdp      6
real_gdp                6
dtype: int64

In [ ]:
# Tail of clean_gdp_df (only geofips column)
clean_gdp_df['geofips'].tail()


19027    98000
19028    98000
19029    98000
19030    98000
19031    98000
Name: geofips, dtype: int64

# Dropped Data

In [21]:
dropped_gdp_path = gs_path + dropped_folder + gdp_folder + 'dropped_raw_gdp_data.csv'
dropped_gdp_df = pd.read_csv(dropped_gdp_path, header=0)

In [22]:
dropped_gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GeoFIPS      18 non-null     object
 1   GeoName      18 non-null     object
 2   Region       18 non-null     int64 
 3   Description  18 non-null     object
 4   2017         18 non-null     object
 5   2018         18 non-null     object
 6   2019         18 non-null     object
 7   2020         18 non-null     object
 8   2021         18 non-null     object
 9   2022         18 non-null     object
 10  drop_reason  18 non-null     object
dtypes: int64(1), object(10)
memory usage: 1.7+ KB


In [9]:
# print the number of rows in the dropped dataframe
dropped_total_rows = len(dropped_gdp_df)
print(f"Total number of rows in dropped dataframe: {dropped_total_rows}")

Total number of rows in dropped dataframe: 18


In [10]:
# Print the number of unique GeoFIPS in the dropped_gdp_df
dropped_unique_geo_fips = dropped_gdp_df['GeoFIPS'].nunique()
print(f"Number of unique GeoFIPS in dropped dataframe: {dropped_unique_geo_fips}")

Number of unique GeoFIPS in dropped dataframe: 6


In [11]:
# Show all the rows in the dropped dataframe
print("Dropped dataframe rows:")
dropped_gdp_df.tail(18)

Dropped dataframe rows:


,GeoFIPS,GeoName,Region,Description,2017,2018,2019,2020,2021,2022,drop_reason
0,"""02063""","Chugach Census Area, AK*",8,Real GDP (thousands of chained 2017 dollars),(NA),(NA),(NA),1268010,1330368,1406228,2017_na
1,"""02063""","Chugach Census Area, AK*",8,Chain-type quantity indexes for real GDP,(NA),(NA),(NA),(NM),(NM),(NM),2017_na
2,"""02063""","Chugach Census Area, AK*",8,Current-dollar GDP (thousands of current dolla...,(NA),(NA),(NA),1291960,1426984,1635885,2017_na
3,"""02066""","Copper River Census Area, AK*",8,Real GDP (thousands of chained 2017 dollars),(NA),(NA),(NA),106221,124946,128311,2017_na
4,"""02066""","Copper River Census Area, AK*",8,Chain-type quantity indexes for real GDP,(NA),(NA),(NA),(NM),(NM),(NM),2017_na
5,"""02066""","Copper River Census Area, AK*",8,Current-dollar GDP (thousands of current dolla...,(NA),(NA),(NA),108227,132650,143508,2017_na
6,"""02201""","Prince of Wales-Outer Ketchikan Census Area, AK*",8,Real GDP (thousands of chained 2017 dollars),(NA),(NA),(NA),(NA),(NA),(NA),2017_na
7,"""02201""","Prince of Wales-Outer Ketchikan Census Area, AK*",8,Chain-type quantity indexes for real GDP,(NA),(NA),(NA),(NA),(NA),(NA),2017_na
8,"""02201""","Prince of Wales-Outer Ketchikan Census Area, AK*",8,Current-dollar GDP (thousands of current dolla...,(NA),(NA),(NA),(NA),(NA),(NA),2017_na
9,"""02232""","Skagway-Hoonah-Angoon Census Area, AK*",8,Real GDP (thousands of chained 2017 dollars),(NA),(NA),(NA),(NA),(NA),(NA),2017_na


# Final Data

In [23]:
dim_geography_path = gs_path + final_folder + 'dim_geography.csv'
dim_geography_df = pd.read_csv(dim_geography_path, header=0)

In [24]:
dim_geography_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   geofips              3172 non-null   int64 
 1   geo_name             3172 non-null   object
 2   region               3172 non-null   object
 3   project_state        3172 non-null   object
 4   project_county_name  3172 non-null   object
dtypes: int64(1), object(4)
memory usage: 124.0+ KB
